In [1]:
import pandas as pd
import numpy as np
from pystoned import CNLS, CNLSDDF, StoNED
from pystoned.constant import CET_MULT, CET_ADDI, FUN_COST, FUN_PROD, RTS_VRS, RTS_CRS, RED_MOM, OPT_LOCAL, RED_QLE, RED_KDE

In [2]:
data = pd.read_csv("../../data_ready/data_ready.csv").dropna()

In [3]:
data_cze = data[data["CNT"] == 'CZE']

inputs_cze = data_cze[["total_students", "ratio_f2m", "ratio_t2s", "ratio_pt2ft"]].values.tolist()
outputs_cze = data_cze[["avg_math", "avg_cog", "avg_science", "avg_read"]]
d_outputs_cze = data_cze[["avg_math", "avg_cog", "avg_science", "avg_read"]].values.tolist()
undesirables_cze = data_cze[["dropout_rate"]].values.tolist()
contextuals_cze = data_cze[["bol_competition", "bol_location"]].values.tolist()
data_cze["avg_scores"] = outputs_cze.mean(axis = 1)
z_output_cze = data_cze[["avg_scores"]].values.tolist()

<ipython-input-3-e170d3ffae16>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cze["avg_scores"] = outputs_cze.mean(axis = 1)


In [4]:
model1_cze = CNLS.CNLS(
    y = z_output_cze, 
    x = inputs_cze, 
    z = contextuals_cze, 
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_CRS
)

model2_cze = CNLS.CNLS(
    y = z_output_cze, 
    x = inputs_cze, 
    z = contextuals_cze, 
    cet = CET_ADDI, 
    fun = FUN_PROD, 
    rts = RTS_VRS
)

# define the CNLSDDF model with undesirable output
model3_cze = CNLSDDF.CNLSDDF(
    y = d_outputs_cze, 
    x = inputs_cze, 
    b = undesirables_cze, 
    gb = -1.0,
    gy = [1.0, 1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)

# define the CNLSDDF model with undesirable output
model4_cze = CNLSDDF.CNLSDDF(
    y = d_outputs_cze, 
    x = inputs_cze, 
    gy = [1.0, 1.0, 1.0, 1.0],
    gx = [1.0, 1.0, 1.0, 1.0],
    fun = FUN_PROD
)

In [7]:
model1_cze.optimize(OPT_LOCAL)
model2_cze.optimize(OPT_LOCAL)
model3_cze.optimize(OPT_LOCAL)
model4_cze.optimize(OPT_LOCAL)

Optimizing locally.
Estimating the additive model locally with mosek solver.
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : QO (quadratic optimization problem)
  Constraints            : 38416           
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 1374            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.03    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 196
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.

In [8]:
### inefficiency
ineff1_cze = StoNED.StoNED(model1_cze)
ineff2_cze = StoNED.StoNED(model2_cze)
ineff3_cze = StoNED.StoNED(model3_cze)
ineff4_cze = StoNED.StoNED(model4_cze)

In [9]:
res_cze = {
    "res1_cze" : {
        "alpha" : "None",
        "beta" : model1_cze.get_beta(),
        "delta" : "None",
        "gamma" : "None",
        "lamda" : model1_cze.get_lamda(),
        "residuals" : model1_cze.get_residual(),
        "frontier" : model1_cze.get_frontier(),
        "unconditional_expected_inefficiency" : ineff1_cze.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff1_cze.get_technical_inefficiency(method = RED_QLE)
    },


    "res2_cze" : {
        "alpha" : model2_cze.get_alpha(),
        "beta" : model2_cze.get_beta(),
        "delta" : "None",
        "gamma" : "None",
        "lamda" : model2_cze.get_lamda(),
        "residuals" : model2_cze.get_residual(),
        "frontier" : model2_cze.get_frontier(),
        "unconditional_expected_inefficiency" : ineff2_cze.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff2_cze.get_technical_inefficiency(method = RED_QLE)
    },


    "res3_cze" : {
        "alpha" : model3_cze.get_alpha(),
        "beta" : model3_cze.get_beta(),
        "delta" : model3_cze.get_delta(),
        "gamma" : model3_cze.get_gamma(),
        "lamda" : "None",
        "residuals" : model3_cze.get_residual(),
        "frontier" : "None",
        "unconditional_expected_inefficiency" : ineff3_cze.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff3_cze.get_technical_inefficiency(method = RED_QLE)
    },


    "res4_cze" : {
        "alpha" : "None",
        "beta" : model4_cze.get_beta(),
        "delta" : "None",
        "gamma" : model4_cze.get_gamma(),
        "lamda" : "None",
        "residuals" : model4_cze.get_residual(),
        "frontier" : "None",
        "unconditional_expected_inefficiency" : ineff4_cze.get_unconditional_expected_inefficiency(method = RED_KDE),
        "technical_inefficiency" : ineff4_cze.get_technical_inefficiency(method = RED_QLE)
    }
}


/home/hsin/miniconda3/envs/papyr2/lib/python3.9/site-packages/pystoned/StoNED.py:62: RuntimeWarning: divide by zero encountered in divide
  return (self.y - Eu) / self.y


In [10]:
import pickle
file = 'res_cze'
open_file = open(file, "wb")

pickle.dump(res_cze, open_file)
open_file.close()


# open_file = open(file, "rb")
# res_cze = pickle.load(open_file)
# open_file.close()

